In [2]:
from main import get_custom_features, add_classic_indicators, get_pair_tickers
import pandas as pd

eth_usdt = pd.read_pickle('usdt_data/ETHUSDT.pickle').astype(float)
eth_usdt['returns'] = eth_usdt['close'].pct_change()
eth_usdt = eth_usdt.dropna()

In [3]:
tickers = get_pair_tickers()

In [ ]:
from tqdm import tqdm
for ticker in tqdm(tickers[30:]):
    data = pd.read_pickle(f'usdt_data/{ticker}.pickle').astype(float)
    data['returns'] = data['close'].pct_change()
    data = data.dropna()
    data = add_classic_indicators(data)
    data = get_custom_features(eth_usdt)
    data.to_pickle(f'usdt_data_features/{ticker}.pickle')

 20%|██        | 79/390 [3:36:06<13:19:37, 154.27s/it]

In [49]:
eth_usdt = get_custom_features(eth_usdt)

In [91]:
test = eth_usdt['returns'].copy()
test[test>0.01] = 1
test[test>0.03] = 2
test[test<=0.01] = 0

In [64]:
import numpy as np

data_fourier = pd.concat([pd.Series(eth_usdt.index.values), eth_usdt['close'].reset_index().iloc[:,1:]], axis=1)
data_fourier.columns = ['date','price']
close_fft = np.fft.fft(np.asarray(data_fourier['price'].tolist()))
fft_df = pd.DataFrame({'fft':close_fft})
fft_df['absolute'] = fft_df['fft'].apply(lambda x: np.abs(x))
fft_df['angle'] = fft_df['fft'].apply(lambda x: np.angle(x))
fft_list = np.asarray(fft_df['fft'].tolist())
for num_ in [2, 7, 15, 100]:
    fft_list_m10= np.copy(fft_list); fft_list_m10[num_:-num_]=0
    data_fourier[f'Fourier {num_}'] = fft_list_m10